<a href="https://colab.research.google.com/github/PandukaBandara99/Database-Engineering/blob/main/Advanced_MySQL_Topics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mahela Panduka Bandara ; <br> UG @ Dpt. of Electronic & Electrical Engineering ;<br> University of Peradeniya ;<br> 2024/03 <br><br> Email : e19039@end.pdn.ac.lk <br> LinkedIN : https://www.linkedin.com/in/pandukabandara/

# **1.0 Functions and More Complex Stored Procedures in MySQL**

## 1.1 Functions vs Procedures

| Aspect              | Functions                                    | Procedures                                       |
|---------------------|----------------------------------------------|--------------------------------------------------|
| Return Value        | Must return a value.                         | May or may not return a value.                   |
| Usage               | Typically used for computations and data manipulation. | Used for executing a series of actions or operations. |
| Call               | Invoked by their name and parameters.        | Invoked by their name and parameters.           |
| Parameters          | Can have input parameters and optionally output parameters. | Can have input and output parameters.            |
| State               | Generally, functions maintain a stateless behavior. | Procedures can have stateful behavior.           |
| Reusability         | Highly reusable as they can be called from multiple places. | Not as reusable since they often perform specific tasks. |
| Invocation          | Can be used within expressions and assignments. | Cannot be used within expressions.              |
| Return Statement    | Must have a return statement specifying the return value. | No explicit return statement is required.        |
| Example (PL/SQL)    | ```sql CREATE FUNCTION calculate_area(length NUMBER, width NUMBER) RETURN NUMBER IS area NUMBER; BEGIN area := length * width; RETURN area; END;``` | ```sql CREATE PROCEDURE insert_record(name VARCHAR2, age NUMBER) IS BEGIN INSERT INTO employee_table(name, age) VALUES(name, age); END;``` |


## 1.2 Declaring Variables

Variables used to pass values between SQL statements or between a procedure and a SQL statement.

Can create
- Inside or outside of a stored procedure
- Inside or outside of a SELECT statement.

### SET - Outside the stored procedure

Syntax: ```SET @variable_name = value;```

Example : <br>
```sql
SET @order_id = 3;
SELECT * FROM Orders WHERE Order_ID = @order_id
```

### DECLARE - Inside the stored procedure

Syntax: ```DECLARE variable_name DATATYPE DEFAULT VALUE;```

Example <br>
```sql
DECLARE minimum_order_cost DECIMAL(5,2) DEFAULT 0;
```

### ASSIGNMENT Operator - Inside SELECT Command

Syntax :  ```SELECT @variable_name := value```

Example:<br>
```sql
SELECT @max_order := MAX(Cost) FROM Orders;
SELECT @max_order;
```

### Variable Inside SELECT Command

Syntax : ``` SELECT function() INTO variable_name FROM table_name;```

Example:<br>
```sql
SELECT AVG(Cost) INTO @average_cost FROM Orders;
```

## 1.3 Parameters

Parameters pass arguments or values to a function or procedure from outside.

- In parameters
- out parameters
- in out parameters

### IN parameter

Syntax: ```CREATE PROCEDURE procedure_name(IN logic(value1, value2)) SELECT logic;```

```sql
CREATE PROCEDURE CalculateTax(IN Salary DECIMAL(10,2)) SELECT Salary *0.2 AS TAX;
CALL CalculateTax(10000);
```

### OUT parameter

Syntax:
```sql
CREATE PROCEDURE GetLowestCost(OUT LowestCost DECIMAL(6,2));
SELECT MIN(Cost) INFO LowestCost FROM Orders;
```

### INOUT parameter

Syntax:
```sql
CREATE PROCEDURE SquareAnumber(INOUT aNumber INT)
BEGIN
  set aNumber = aNumber*aNumber;
END
```

```sql
SET @x_number = 5
CALL SquareAnumber (@x_number);
```

## 1.4 User Defined Functions

Syntax <br>
```sql
CREATE FUNCTION function_name()
RETURNS datatype DETERMINISTIC
RETURN
```

Example: <br>
```sql
CREATE FUNCTION FindTotalCost(Cost DECIMAL(5,2))
RETURNS DECIMAL (5,2) DETERMINISTIC
RETURN (Cost - (Cost*0.1));
```

```sql
SELECT FindTotalCost(100)
```

Example: Apply 10% Discount Purchases  between 100$ and 500$ , 20% for purchases above 500$.<br>
```sql
CREATE FUNCTION FindTotalCost(Cost DECIMAL(5,2))
RETURNS DECIMAL (5,2) DETERMINISTIC
  BEGIN IF (Cost >= 100 AND Cost <500) THEN SET Cost  = Cost-(Cost*0.1);
        ELSEIF (Cost >= 500) THEN SET Cost = Cost-(Cost*0.2);
        END IF;
RETURN Cost;
```

## 1.5 Complex Stored Procedures

Consider <br>
```sql
+----+-------------+----------+------------+
| id | name        | salary   | department |
+----+-------------+----------+------------+
| 1  | John Doe    | 50000.00 | Engineering|
| 2  | Jane Smith  | 60000.00 | HR         |
| 3  | Mike Johnson| 55000.00 | Marketing  |
+----+-------------+----------+------------+
```

```sql
DELIMITER //

CREATE PROCEDURE update_salaries (IN dept VARCHAR(100), IN increase DECIMAL(5, 2))
BEGIN
    DECLARE done INT DEFAULT FALSE;
    DECLARE emp_id INT;
    DECLARE emp_salary DECIMAL(10, 2);
    DECLARE emp_cursor CURSOR FOR
        SELECT id, salary FROM employees WHERE department = dept;
    DECLARE CONTINUE HANDLER FOR NOT FOUND SET done = TRUE;

    OPEN emp_cursor;

    emp_loop: LOOP
        FETCH emp_cursor INTO emp_id, emp_salary;
        IF done THEN
            LEAVE emp_loop;
        END IF;
        
        -- Update the salary of the employee
        UPDATE employees SET salary = emp_salary * (1 + increase / 100)
        WHERE id = emp_id;
    END LOOP;

    CLOSE emp_cursor;
END//

DELIMITER ;
```

In this stored procedure:

- We first declare some variables (done, emp_id, emp_salary) to use within the procedure.
- We declare a cursor named emp_cursor that selects all employees from the specified department.
- We define a continue handler for the cursor to set the done variable to true when there are no more rows to fetch.
- We open the cursor and iterate over each employee using a loop.
- Inside the loop, we fetch the employee ID and salary, then update the salary based on the given percentage increase for the department.
- Finally, we close the cursor.